# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 8:30AM,258091,10,0086428397,ISDIN Corporation,Released
1,Feb 24 2023 8:30AM,258090,10,0086428205,ISDIN Corporation,Released
2,Feb 24 2023 8:30AM,258090,10,0086428206,ISDIN Corporation,Released
3,Feb 24 2023 8:30AM,258090,10,0086428207,ISDIN Corporation,Released
4,Feb 24 2023 8:30AM,258091,10,0086427907,ISDIN Corporation,Released
5,Feb 24 2023 8:30AM,258091,10,0086427906,ISDIN Corporation,Released
6,Feb 24 2023 8:30AM,258091,10,0086428401,ISDIN Corporation,Released
7,Feb 24 2023 8:30AM,258090,10,0086427466,ISDIN Corporation,Released
8,Feb 24 2023 8:30AM,258090,10,0086427464,ISDIN Corporation,Released
9,Feb 24 2023 8:30AM,258090,10,0086427482,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,258090,Released,56
30,258091,Released,4
31,258094,Released,1
32,258095,Released,1
33,258097,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258090,NaN,NaN,56.0
258091,NaN,NaN,4.0
258094,NaN,NaN,1.0
258095,NaN,NaN,1.0
258097,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257988,0.0,1.0,1.0
258014,14.0,1.0,2.0
258020,0.0,1.0,0.0
258031,4.0,2.0,1.0
258039,5.0,13.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257988,0,1,1
258014,14,1,2
258020,0,1,0
258031,4,2,1
258039,5,13,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257988,0,1,1
1,258014,14,1,2
2,258020,0,1,0
3,258031,4,2,1
4,258039,5,13,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257988,,1,1
1,258014,14,1,2
2,258020,,1,
3,258031,4,2,1
4,258039,5,13,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation
55,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc."
65,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc."
66,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc."
67,Feb 23 2023 3:41PM,258097,22,"NBTY Global, Inc."
68,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences
69,Feb 23 2023 2:37PM,258081,10,ISDIN Corporation
74,Feb 23 2023 2:30PM,258080,19,"AdvaGen Pharma, Ltd"
92,Feb 23 2023 2:28PM,258079,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,,,4
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,,,56
2,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",,,5
3,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",,,1
4,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",,,1
5,Feb 23 2023 3:41PM,258097,22,"NBTY Global, Inc.",,,1
6,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences,,,1
7,Feb 23 2023 2:37PM,258081,10,ISDIN Corporation,,,5
8,Feb 23 2023 2:30PM,258080,19,"AdvaGen Pharma, Ltd",,,18
9,Feb 23 2023 2:28PM,258079,10,ISDIN Corporation,,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
2,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
3,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",1,,
4,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",1,,
5,Feb 23 2023 3:41PM,258097,22,"NBTY Global, Inc.",1,,
6,Feb 23 2023 3:21PM,258087,19,Opus Life Sciences,1,,
7,Feb 23 2023 2:37PM,258081,10,ISDIN Corporation,5,,
8,Feb 23 2023 2:30PM,258080,19,"AdvaGen Pharma, Ltd",18,,
9,Feb 23 2023 2:28PM,258079,10,ISDIN Corporation,12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
2,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
3,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",1,,
4,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,NaN,NaN
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,NaN,NaN
2,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,NaN,NaN
3,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,0.0,0.0
1,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,0.0,0.0
2,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,0.0,0.0
3,Feb 23 2023 3:45PM,258095,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 23 2023 3:43PM,258094,19,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5161063,174.0,19.0,23.0
12,258044,1.0,0.0,0.0
19,1032332,35.0,0.0,0.0
20,258052,1.0,0.0,0.0
22,516192,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5161063,174.0,19.0,23.0
1,12,258044,1.0,0.0,0.0
2,19,1032332,35.0,0.0,0.0
3,20,258052,1.0,0.0,0.0
4,22,516192,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,174.0,19.0,23.0
1,12,1.0,0.0,0.0
2,19,35.0,0.0,0.0
3,20,1.0,0.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,174.0
1,12,Released,1.0
2,19,Released,35.0
3,20,Released,1.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Completed,23.0,0.0,0.0,0.0,0.0
Executing,19.0,0.0,0.0,0.0,0.0
Released,174.0,1.0,35.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Completed,23.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,0.0,0.0
2,Released,174.0,1.0,35.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Completed,23.0,0.0,0.0,0.0,0.0
1,Executing,19.0,0.0,0.0,0.0,0.0
2,Released,174.0,1.0,35.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()